In [4]:
import json
import os

def analisar_top_verbetes():
    """
    Analisa um arquivo JSON de verbetes para extrair os 20 principais
    verbetes com base em várias métricas de rede.
    O script espera que o arquivo JSON de entrada esteja em uma subpasta.
    """
    # --- Configuração de Caminhos ---
    input_dir = "../dados"
    input_filename = "dados_com_constraint.json"
    input_path = os.path.join(input_dir, input_filename)
    
    output_dir = "../dados/IndicadoresGerais"
    output_filename = "TopVerbetes.json"
    output_path = os.path.join(output_dir, output_filename)

    # --- 1. Carregar o arquivo JSON de entrada ---
    try:
        with open(input_path, 'r', encoding='utf-8') as f:
            dados = json.load(f)
        # A lista de verbetes está sob a chave 'verbetes_completo'
        verbetes = dados.get('verbetes_completo', [])
        if not verbetes:
            print(f"Erro: A chave 'verbetes_completo' não foi encontrada ou está vazia no arquivo '{input_path}'.")
            return
            
        print(f"Arquivo '{input_path}' carregado com sucesso. {len(verbetes)} verbetes encontrados.")
    except FileNotFoundError:
        print(f"Erro: O arquivo de entrada '{input_path}' não foi encontrado.")
        return
    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{input_path}' não é um JSON válido.")
        return

    # --- 2. Definir as métricas para análise ---
    metricas = [
        "total_degree",
        "betweenness_centrality",
        "pagerank",
        "clustering_coefficient",
        "closeness_centrality",
        "metrica_composta",
        "constraint",
        "quantidade_edicoes"
    ]

    resultados_finais = {}

    # --- 3. Processar cada métrica ---
    print("\nIniciando a análise das métricas...")
    for metrica in metricas:
        print(f"Processando '{metrica}'...")
        
        # Filtra verbetes que possuem a métrica
        verbetes_com_metrica = [v for v in verbetes if metrica in v]

        # ATUALIZAÇÃO: Condição especial para clustering_coefficient
        if metrica == "clustering_coefficient":
            verbetes_com_metrica = [v for v in verbetes_com_metrica if v[metrica] < 1.0]

        # Ordena os verbetes pela métrica em ordem decrescente
        verbetes_ordenados = sorted(verbetes_com_metrica, key=lambda x: x[metrica], reverse=True)

        # Seleciona os 20 primeiros
        top_20 = verbetes_ordenados[:20]

        # ATUALIZAÇÃO: Formata o resultado para incluir os novos atributos
        resultados_finais[f"top_20_{metrica}"] = [
            {
                "titulo": verbete.get("titulo"),
                metrica: verbete.get(metrica),
                "quantidade_edicoes": verbete.get("quantidade_edicoes"),
                "community_id": verbete.get("community_id"),
                "data_criacao": verbete.get("data_criacao"),
                "data_ultima_edicao": verbete.get("data_ultima_edicao")
            }
            for verbete in top_20
        ]
    
    print("\nAnálise concluída.")

    # --- 4. Salvar o resultado em um novo arquivo JSON ---
    try:
        # Cria o diretório de saída se ele não existir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"Diretório '{output_dir}' criado.")

        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(resultados_finais, f, ensure_ascii=False, indent=4)
        
        print(f"\n✅ Resultados salvos com sucesso em '{output_path}'!")

    except Exception as e:
        print(f"\nOcorreu um erro ao salvar o arquivo: {e}")


if __name__ == '__main__':
    # Este script assume que a pasta 'dadosWikifavelas20250511' 
    # está no mesmo diretório que ele.
    analisar_top_verbetes()


Arquivo '../dados\dados_com_constraint.json' carregado com sucesso. 3552 verbetes encontrados.

Iniciando a análise das métricas...
Processando 'total_degree'...
Processando 'betweenness_centrality'...
Processando 'pagerank'...
Processando 'clustering_coefficient'...
Processando 'closeness_centrality'...
Processando 'metrica_composta'...
Processando 'constraint'...
Processando 'quantidade_edicoes'...

Análise concluída.

✅ Resultados salvos com sucesso em '../dados/IndicadoresGerais\TopVerbetes.json'!


In [5]:
# visualizar_metrica_composta.py
#
# Este script realiza uma análise visual detalhada dos 20 verbetes mais
# importantes da rede, com base na métrica composta.
#
# Como especialista em visualização de dados, a abordagem é:
# 1. Isolar os dados dos atores-chave (top 20 verbetes).
# 2. Para cada ator, decompor a métrica composta em suas partes constituintes.
# 3. Utilizar uma biblioteca de visualização de alta qualidade (Plotly.js) para
#    gerar um dashboard interativo com um seletor de verbetes.
# 4. O design (cores, layout) é consistente para facilitar a análise
#    e a extração de insights.
# 5. ATUALIZAÇÃO: Adiciona o ranking global de cada métrica como um label dentro
#    das barras do gráfico para uma análise comparativa mais rica.

import json
from pathlib import Path
import pandas as pd

def gerar_visualizacao_composicao_metrica():
    """
    Gera um dashboard HTML com um menu suspenso para selecionar e visualizar
    o gráfico de barras de composição da métrica para cada um dos 20 principais verbetes.
    """
    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('../dados')
    INPUT_DIR_2 = Path('../dados/IndicadoresGerais')

    
    RANKINGS_FILENAME = 'TopVerbetes.json'
    FULL_DATA_FILENAME = 'dados_com_metrica_composta.json'
    
    OUTPUT_DIR = Path('../documentos')
    OUTPUT_DIR.mkdir(exist_ok=True)
    OUTPUT_FILENAME = 'dashboard_interativo_composicao_metrica.html'

    rankings_path = INPUT_DIR_2 / RANKINGS_FILENAME
    full_data_path = INPUT_DIR / FULL_DATA_FILENAME
    output_path = OUTPUT_DIR / OUTPUT_FILENAME

    # --- 2. CARREGAMENTO E PROCESSAMENTO DOS DADOS ---
    print("Carregando e processando os dados...")
    try:
        with open(rankings_path, 'r', encoding='utf-8') as f:
            rankings_data = json.load(f)
        
        with open(full_data_path, 'r', encoding='utf-8') as f:
            full_data = json.load(f)

        top_20_verbetes_info = rankings_data.get('top_20_metrica_composta', [])
        top_20_titulos = [verbete['titulo'] for verbete in top_20_verbetes_info]

        df_full = pd.DataFrame(full_data.get('verbetes_completo', []))
        
        # --- ATUALIZAÇÃO: CALCULAR RANKING GLOBAL PARA CADA MÉTRICA ---
        print("Calculando rankings globais para cada métrica...")
        metricas_base = ['betweenness_centrality', 'closeness_centrality', 'total_degree', 'quantidade_edicoes', 'pagerank', 'metrica_composta']
        for metrica in metricas_base:
            # .rank() atribui a posição a cada linha. method='min' lida com empates.
            df_full[f'{metrica}_rank'] = df_full[metrica].rank(method='min', ascending=False).astype(int)

        df_top_20 = df_full[df_full['titulo'].isin(top_20_titulos)].copy()
        df_top_20['titulo'] = pd.Categorical(df_top_20['titulo'], categories=top_20_titulos, ordered=True)
        df_top_20.sort_values('titulo', inplace=True)

    except FileNotFoundError as e:
        print(f"ERRO: Não foi possível encontrar o arquivo de entrada: {e.filename}")
        return
    except Exception as e:
        print(f"Ocorreu um erro ao carregar os dados: {e}")
        return

    # --- 3. PREPARAÇÃO DOS DADOS PARA A VISUALIZAÇÃO ---
    print("Preparando dados para a visualização Plotly...")
    
    metricas_plot = [
        'betweenness_centrality_norm', 'closeness_centrality_norm',
        'total_degree_norm', 'quantidade_edicoes_norm', 'pagerank_norm',
        'metrica_composta'
    ]
    
    metric_labels = {
        'betweenness_centrality_norm': 'Betweenness', 'closeness_centrality_norm': 'Closeness',
        'total_degree_norm': 'Degree', 'quantidade_edicoes_norm': 'Edições',
        'pagerank_norm': 'PageRank', 'metrica_composta': 'Métrica Composta'
    }
    
    # Mapeia as métricas normalizadas para as suas colunas de ranking correspondentes
    metric_to_rank_col = {
        'betweenness_centrality_norm': 'betweenness_centrality_rank',
        'closeness_centrality_norm': 'closeness_centrality_rank',
        'total_degree_norm': 'total_degree_rank',
        'quantidade_edicoes_norm': 'quantidade_edicoes_rank',
        'pagerank_norm': 'pagerank_rank',
        'metrica_composta': 'metrica_composta_rank'
    }

    data_for_js = []
    for index, row in df_top_20.iterrows():
        verbete_data = {'titulo': row['titulo']}
        verbete_data['x_values'] = [metric_labels[m] for m in metricas_plot]
        verbete_data['y_values'] = [row.get(m, 0) for m in metricas_plot]
        # ATUALIZAÇÃO: Adiciona os textos de ranking
        verbete_data['text_values'] = [f"#{row.get(metric_to_rank_col[m], '')}" for m in metricas_plot]
        data_for_js.append(verbete_data)

    data_json = json.dumps(data_for_js, ensure_ascii=False)

    # --- 4. GERAÇÃO DO ARQUIVO HTML ---
    print("Gerando o arquivo HTML do dashboard interativo...")

    html_content = f"""
    <!DOCTYPE html>
    <html lang="pt-br">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Dashboard Interativo da Métrica Composta - Top 20 Verbetes</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{
                font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif;
                background-color: #0d1117;
                color: #c9d1d9;
                margin: 0;
                padding: 2rem;
            }}
            h1 {{
                text-align: center;
                color: #f0f6fc;
                padding-bottom: 1rem;
                margin-bottom: 1rem;
            }}
            .dashboard-controls {{
                text-align: center;
                margin-bottom: 2rem;
                border-bottom: 1px solid #30363d;
                padding-bottom: 2rem;
            }}
            #verbete-selector {{
                background-color: #161b22;
                color: #c9d1d9;
                border: 1px solid #30363d;
                padding: 0.5rem 1rem;
                border-radius: 6px;
                font-size: 1rem;
                cursor: pointer;
            }}
            #chart-container {{
                background-color: #161b22;
                border: 1px solid #30363d;
                border-radius: 8px;
                padding: 1rem;
                width: 100%;
                max-width: 900px;
                margin: 0 auto;
            }}
        </style>
    </head>
    <body>
        <h1>Análise da Métrica Composta: Top 20 Verbetes</h1>
        
        <div class="dashboard-controls">
            <label for="verbete-selector" style="margin-right: 10px;">Selecione um Verbete:</label>
            <select id="verbete-selector"></select>
        </div>

        <div id="chart-container"></div>

        <script>
            const plotData = {data_json};

            const barColors = ['#58a6ff', '#3fb950', '#e3b341', '#f778ba', '#a371f7', '#ff7b72'];

            const layoutTemplate = {{
                paper_bgcolor: '#161b22',
                plot_bgcolor: '#161b22',
                font: {{ color: '#c9d1d9' }},
                margin: {{ l: 50, r: 20, b: 100, t: 80 }},
                xaxis: {{ tickfont: {{ size: 12 }} }},
                yaxis: {{ gridcolor: '#30363d' }}
            }};

            const selector = document.getElementById('verbete-selector');
            const chartContainer = document.getElementById('chart-container');

            // Popula o menu suspenso
            plotData.forEach((verbete, index) => {{
                const option = document.createElement('option');
                option.value = index;
                option.textContent = `${{index + 1}}. ${{verbete.titulo}}`;
                selector.appendChild(option);
            }});

            // Função para desenhar/atualizar o gráfico
            function displayChart(index) {{
                const verbete = plotData[index];
                
                // ATUALIZAÇÃO: Adiciona os textos de ranking ao gráfico
                const data = [{{
                    x: verbete.x_values,
                    y: verbete.y_values,
                    text: verbete.text_values, // Array com os rankings
                    textposition: 'inside',
                    textfont: {{
                        color: '#161b22', // Cor escura para contrastar com a barra
                        size: 12,
                        family: 'Arial, sans-serif',
                        weight: 'bold'
                    }},
                    type: 'bar',
                    marker: {{ color: barColors }}
                }}];
                
                let layout = JSON.parse(JSON.stringify(layoutTemplate));
                layout.title = {{
                    text: `<b>${{verbete.titulo}}</b>`,
                    font: {{ size: 18 }}
                }};

                Plotly.newPlot(chartContainer, data, layout, {{responsive: true}});
            }}

            // Adiciona o listener para o evento de mudança
            selector.addEventListener('change', (event) => {{
                displayChart(event.target.value);
            }});

            // Exibe o primeiro gráfico ao carregar a página
            displayChart(0);
        </script>
    </body>
    </html>
    """

    # --- 5. SALVANDO O ARQUIVO FINAL ---
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
        
    print(f"\n✅ Dashboard interativo gerado com sucesso em '{output_path}'!")
    print("Abra este arquivo em um navegador para ver os gráficos.")

if __name__ == '__main__':
    gerar_visualizacao_composicao_metrica()


Carregando e processando os dados...
Calculando rankings globais para cada métrica...
Preparando dados para a visualização Plotly...
Gerando o arquivo HTML do dashboard interativo...

✅ Dashboard interativo gerado com sucesso em '..\documentos\dashboard_interativo_composicao_metrica.html'!
Abra este arquivo em um navegador para ver os gráficos.


In [7]:
# gerar_indicadores_wikifavelas.py
#
# Este script lê o arquivo JSON completo com os dados dos verbetes
# e calcula um conjunto de indicadores agregados, como os principais
# editores, referências, categorias e os verbetes mais editados.
# O resultado é salvo em um arquivo JSON de resumo.

import json
from pathlib import Path
from collections import Counter

def gerar_arquivo_de_indicadores():
    """
    Calcula e salva os principais indicadores da rede Wikifavelas.
    """
    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('../dados')
    OUTPUT_DIR = Path('../dados/IndicadoresGerais')
    # Use o nome do arquivo gerado pelo seu script de coleta
    INPUT_FILENAME = 'dados_com_metrica_composta.json' 
    OUTPUT_FILENAME = 'indicadoresWikifavelas.json'
    
    input_path = INPUT_DIR / INPUT_FILENAME
    output_path = OUTPUT_DIR / OUTPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de entrada não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ---
    print(f"Carregando dados de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes = dados.get('verbetes_completo', [])
    if not verbetes:
        print("Nenhum verbete encontrado no arquivo.")
        return

    # --- 3. CÁLCULO DOS INDICADORES ---
    print("Calculando os indicadores agregados...")

    # a) Top 20 Editores
    editores_counter = Counter()
    for verbete in verbetes:
        for editor, count in verbete.get('usuarios_edicoes', {}).items():
            editores_counter[editor] += count
    
    top_editores = [
        {"editor": editor, "quantidade_edicoes": count} 
        for editor, count in editores_counter.most_common(20)
    ]
    print(" - Top 20 editores calculado.")

    # b) Top 20 Referências (verbetes mais citados)
    referencias_counter = Counter()
    for verbete in verbetes:
        for ref in verbete.get('referencias', []):
            referencias_counter[ref] += 1
            
    top_referencias = [
        {"referencia": ref, "quantidade": count}
        for ref, count in referencias_counter.most_common(20)
    ]
    print(" - Top 20 referências calculado.")

    # c) Top 20 Categorias
    categorias_counter = Counter()
    for verbete in verbetes:
        for cat in verbete.get('categorias', []):
            categorias_counter[cat] += 1

    top_categorias = [
        {"categoria": cat, "quantidade": count}
        for cat, count in categorias_counter.most_common(20)
    ]
    print(" - Top 20 categorias calculado.")

    # d) Top 20 Verbetes Mais Editados
    verbetes_ordenados_por_edicao = sorted(
        verbetes,
        key=lambda v: v.get('quantidade_edicoes', 0),
        reverse=True
    )
    
    top_verbetes_mais_editados = [
        {"titulo": v['titulo'], "quantidade_edicoes": v.get('quantidade_edicoes', 0)}
        for v in verbetes_ordenados_por_edicao[:20]
    ]
    print(" - Top 20 verbetes mais editados calculado.")

    # --- 4. MONTAGEM E SALVAMENTO DO ARQUIVO FINAL ---
    resultado_final = {
        "top_editores": top_editores,
        "top_referencias": top_referencias,
        "top_categorias": top_categorias,
        "top_verbetes_mais_editados": top_verbetes_mais_editados
    }

    print(f"\nSalvando os indicadores em '{output_path}'...")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(resultado_final, f, ensure_ascii=False, indent=4)
        
    print(f"✅ Arquivo de indicadores gerado com sucesso!")

if __name__ == '__main__':
    gerar_arquivo_de_indicadores()


Carregando dados de '..\dados\dados_com_metrica_composta.json'...
Calculando os indicadores agregados...
 - Top 20 editores calculado.
 - Top 20 referências calculado.
 - Top 20 categorias calculado.
 - Top 20 verbetes mais editados calculado.

Salvando os indicadores em '..\dados\IndicadoresGerais\indicadoresWikifavelas.json'...
✅ Arquivo de indicadores gerado com sucesso!


In [8]:
# gerar_graficos_ranking.py
#
# Este script lê o arquivo JSON com os rankings dos top verbetes e gera
# gráficos de barras verticais em formato de imagem (.png) para cada métrica,
# prontos para serem inseridos em uma apresentação.

import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def generate_ranking_plot(data, metric_key, title, output_path):
    """
    Gera e salva um gráfico de barras vertical estilizado para um ranking de métrica específico.
    """
    
    if not data:
        print(f"  - AVISO: Não há dados para a métrica '{metric_key}'. Gráfico não gerado.")
        return

    # Prepara os dados usando a biblioteca pandas
    df = pd.DataFrame(data)
    df = df.head(10) # Garante que estamos a plotar apenas o top 10
    
    if metric_key not in df.columns:
        print(f"  - ERRO: A chave da métrica '{metric_key}' não foi encontrada nos dados. Gráfico não gerado.")
        return

    # ATUALIZAÇÃO: Ordena do maior para o menor para o eixo X
    df.sort_values(by=metric_key, ascending=False, inplace=True)
    
    # Limita o tamanho dos títulos para não sobrecarregar o eixo X
    df['titulo_curto'] = df['titulo'].apply(lambda x: x[:40] + '...' if len(x) > 40 else x)

    # --- Plotando com Seaborn para alta qualidade visual ---
    plt.style.use('dark_background')
    fig, ax = plt.subplots(figsize=(14, 8))

    # ATUALIZAÇÃO: Cria o gráfico de barras vertical com uma paleta de cores
    barplot = sns.barplot(x='titulo_curto', y=metric_key, data=df, palette='viridis', ax=ax)

    # --- Ajustes de Estilo para uma Aparência Profissional ---
    ax.set_title(title, fontsize=22, fontweight='bold', pad=20)
    ax.set_ylabel('Valor da Métrica', fontsize=14, labelpad=15)
    ax.set_xlabel('') # O título de cada barra já serve como rótulo
    
    # ATUALIZAÇÃO: Rotaciona os rótulos do eixo X para garantir a legibilidade
    ax.tick_params(axis='x', labelsize=11)
    plt.xticks(rotation=45, ha='right')
    
    ax.grid(axis='y', linestyle='--', alpha=0.3)
    
    # Adiciona os valores numéricos no topo de cada barra
    ax.bar_label(barplot.containers[0], fmt='%.4f', padding=3, fontsize=10, color='white')

    # Ajusta os limites do eixo Y para dar espaço aos rótulos
    ax.set_ylim(top=ax.get_ylim()[1] * 1.1)

    # Garante que o layout não corte os títulos e salva a figura
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close(fig) # Fecha a figura para libertar memória
    print(f"  - Gráfico salvo em: {output_path}")


def main():
    """
    Função principal que orquestra a geração de todos os gráficos de ranking.
    """
    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('../dados/IndicadoresGerais')
    RANKINGS_FILENAME = 'TopVerbetes.json'
    OUTPUT_DIR = Path('../public/outros')
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    rankings_path = INPUT_DIR / RANKINGS_FILENAME

    if not rankings_path.exists():
        print(f"ERRO: O arquivo de rankings '{rankings_path}' não foi encontrado.")
        return

    # --- 2. CARREGAMENTO DOS DADOS ---
    print(f"Carregando dados de '{rankings_path}'...")
    with open(rankings_path, 'r', encoding='utf-8') as f:
        rankings_data = json.load(f)

    # --- 3. DEFINIÇÃO DAS MÉTRICAS A SEREM PLOTADAS ---
    metrics_to_plot = {
        'Degree': ('top_20_total_degree', 'total_degree'),
        'Betweenness': ('top_20_betweenness_centrality', 'betweenness_centrality'),
        'PageRank': ('top_20_pagerank', 'pagerank'),
        'Closeness': ('top_20_closeness_centrality', 'closeness_centrality'),
        'Constraint': ('top_20_constraint', 'constraint'),
        'Métrica Composta': ('top_20_metrica_composta', 'metrica_composta')
    }

    print("\nGerando gráficos de ranking...")
    for chart_title, (json_key, metric_key) in metrics_to_plot.items():
        print(f"\nProcessando: {chart_title}")
        
        if json_key in rankings_data:
            data = rankings_data[json_key]
            output_filepath = OUTPUT_DIR / f"ranking_{chart_title.lower().replace(' ', '_')}.png"
            generate_ranking_plot(data, metric_key, f"Top 10 Verbetes por {chart_title}", output_filepath)
        else:
            print(f"  - AVISO: A chave '{json_key}' não foi encontrada no arquivo JSON. Gráfico não gerado.")
    
    print("\n✅ Processo concluído!")

if __name__ == '__main__':
    main()


Carregando dados de '..\dados\IndicadoresGerais\TopVerbetes.json'...

Gerando gráficos de ranking...

Processando: Degree


C:\Users\GlauberGuimarãesBati\AppData\Local\Temp\ipykernel_23348\2215242623.py:41: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(x='titulo_curto', y=metric_key, data=df, palette='viridis', ax=ax)


  - Gráfico salvo em: ..\public\outros\ranking_degree.png

Processando: Betweenness


C:\Users\GlauberGuimarãesBati\AppData\Local\Temp\ipykernel_23348\2215242623.py:41: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(x='titulo_curto', y=metric_key, data=df, palette='viridis', ax=ax)


  - Gráfico salvo em: ..\public\outros\ranking_betweenness.png

Processando: PageRank


...

  - Gráfico salvo em: ..\public\outros\ranking_closeness.png

Processando: Constraint


C:\Users\GlauberGuimarãesBati\AppData\Local\Temp\ipykernel_23348\2215242623.py:41: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(x='titulo_curto', y=metric_key, data=df, palette='viridis', ax=ax)


  - Gráfico salvo em: ..\public\outros\ranking_constraint.png

Processando: Métrica Composta


C:\Users\GlauberGuimarãesBati\AppData\Local\Temp\ipykernel_23348\2215242623.py:41: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(x='titulo_curto', y=metric_key, data=df, palette='viridis', ax=ax)


  - Gráfico salvo em: ..\public\outros\ranking_métrica_composta.png

✅ Processo concluído!
